In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import os
import re
import time
import pandas as pd
from pathlib import Path

### Open CSV

In [37]:
pd.read_csv('/home/siebenschuh/Projects/dataprep/data/arxiv/csv/2408.02671v1.csv', sep='|')

,id,title,pdf_url,html_url,summary,comment,date_published,date_updated,doi,prim_cat,categories,journal_ref
0,http://arxiv.org/abs/2408.02671v1,Systematic bias due to mismodelling precessing...,http://arxiv.org/pdf/2408.02671v1.pdf,http://arxiv.org/html/2408.02671v1,Accurate waveform modelling is crucial for par...,"16 pages, 5 figures",2024-08-05 17:59:45+00:00,2024-08-05 17:59:45+00:00,NaN,gr-qc,NaN,NaN


## Open PDF

 \checkmark

## Open HTML

\checkmark

In [31]:
# Extraction pipeline
def extract_abstract(web_content, sec_to_timeout=1):
    '''Extracts `abstract` text from HTML document
    '''
    soup = BeautifulSoup(web_content, 'html.parser')
    abstract_div = soup.find(id="abstract")
    if abstract_div:
        abstract_text = abstract_div.text.strip()
    else:
        abstract_text = None
    return post_process_abstract(abstract_text)

def post_process_abstract(s):
    prefix = "AbstractReport issue for preceding element"
    suffix = "Report issue for preceding element"
    
    if s.startswith(prefix):
        s = s[len(prefix):].strip()  # Remove prefix and strip any leading whitespace
    if s.endswith(suffix):
        s = s[:-len(suffix)].strip()  # Remove suffix and strip any trailing whitespace
    
    return s

def download_plain_text_from_html(web_content):
    soup = BeautifulSoup(web_content, 'html.parser')
    text = soup.get_text()
    return text

def beautify_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    
    replacements = {
        r'\\xa0': ' ',
        r'\\u2062': '',
        r'\\n': '\n',
        r'\\t': '\t',
    }
    
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    
    text = re.sub(r'\\', '', text)
    
    return text

In [32]:
download_dir = Path('/home/siebenschuh/Projects/dataprep/data/arxiv')

# Driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

for html_path in os.listdir(download_dir / 'html'):
    file_path = download_dir / 'html' / html_path
    if file_path.is_file():
        driver.get(f"file://{file_path.resolve()}")
        page_content = driver.page_source
        print(beautify_text(download_plain_text_from_html(page_content)))

        #break
        print('- - - - - - -')
    

Local ergotropy and its fluctuations across a dissipative quantum phase transition Back to arXiv This is experimental HTML to improve accessibility. We invite you to report rendering errors. Use Alt+Y to toggle on accessible reporting links and Alt+Shift+Y to toggle off. Learn more about this project and help improve conversions. Why HTML? Report Issue Back to Abstract Download PDF Table of Contents Abstract I Introduction II The model II.1 Hamiltionian description II.1.1 Phase diagram of the model II.2 Charging, storage and work extraction II.2.1 Discharged state II.2.2 Charging II.2.3 Storage II.2.4 Local work extraction III Numerical Results III.1 Charging capability III.2 Storage and work extraction features III.2.1 Agnostic approach for work extraction III.2.2 Optimized work extraction IV Conclusions V Acknowledgements A Charging two-qubit gate implementation B Quasiprobability distribution approach C Switch-off protocols D Variability of optimized ergotropy dataset E Bayesian opt

In [ ]:
download_plain_text_from_html()

In [13]:
file_path

PosixPath('/home/siebenschuh/Projects/dataprep/data/arxiv/html/2408.02655v1.html')